
# Estudo de Caso – Predição de Falhas em Máquinas (Manutenção Preditiva)
Professor Rodrigo – UniFECAF  
Disciplina: Machine Learning e Deep Learning  
Formato: dupla ou trio • Tempo total estimado: ~2 horas

## Contexto
O Datacenter UniFECAF opera servidores e dispositivos de rede críticos. Falhas inesperadas geram indisponibilidade, perda de dados e custo.
Seu time foi contratado para construir um piloto de manutenção preditiva usando Machine Learning, prevendo falhas a partir de dados de sensores.

Objetivo: construir um pipeline simples e funcional para prever falhas (classificação binária), priorizando recall (reduzir falsos negativos).



## Dataset (download e upload manual)
Usaremos o dataset público AI4I 2020 Predictive Maintenance (UCI Repository).

Baixe o CSV no site oficial da UCI:  
https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset

Após baixar no seu computador, faça o upload do arquivo .csv na célula abaixo.


In [ ]:

# Upload do arquivo CSV para o Colab
from google.colab import files
import io, pandas as pd

print("Selecione o arquivo CSV baixado do site da UCI (ex.: ai4i2020.csv)")
uploaded = files.upload()

# Pega o primeiro arquivo enviado
filename = next(iter(uploaded))
df = pd.read_csv(io.BytesIO(uploaded[filename]))

# Normaliza cabeçalhos
df.columns = [c.strip() for c in df.columns]

print("Arquivo carregado:", filename)
df.head()



## Revisão rápida – Pipeline de Machine Learning
1. Coleta de dados → 2. Exploração (EDA) → 3. Pré-processamento → 4. Treinamento → 5. Avaliação → 6. Recomendação

Pergunta (responda em texto): Em qual etapa você imagina que gastaremos mais tempo hoje e por quê?


Escreva sua resposta aqui nesta célula


## Exploração dos Dados (EDA)
Vamos entender o conjunto de dados, tipos de colunas e estatísticas básicas.


In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:
list(df.columns)


### Checagem das colunas necessárias
Usaremos apenas as colunas abaixo (sem vazamento de informação):
- Type
- Air temperature [K]
- Process temperature [K]
- Rotational speed [rpm]
- Torque [Nm]
- Tool wear [min]
- Alvo: Machine failure (0 = normal, 1 = falha)

Execute a célula de validação a seguir.


In [ ]:

# Validação de colunas esperadas
expected = [
    'Type',
    'Air temperature [K]',
    'Process temperature [K]',
    'Rotational speed [rpm]',
    'Torque [Nm]',
    'Tool wear [min]',
    'Machine failure'
]

present = set(df.columns)
missing = [c for c in expected if c not in present]

if missing:
    print("Colunas ausentes no seu CSV:", missing)
    print("Verifique se você baixou o arquivo correto do site da UCI.")
else:
    print("Todas as colunas necessárias estão presentes.")


### Proporção de falhas

In [ ]:

# Proporção das classes (0 = normal, 1 = falha)
prop = df['Machine failure'].value_counts(normalize=True).rename({0: 'normal', 1: 'falha'})
print("Proporção das classes:")
display(prop)



Perguntas rápidas:  
1) O dataset está balanceado? Por que isso importa?  
2) Qual métrica você imagina ser mais importante aqui: accuracy ou recall? Explique.


Escreva suas respostas aqui


## Pré-processamento
- Selecionar features seguras (sem TWF/HDF/PWF/OSF/RNF).  
- One-Hot Encoding em Type.  
- Split treino/teste estratificado para manter a proporção de classes.


In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split

features = [
    'Type',
    'Air temperature [K]',
    'Process temperature [K]',
    'Rotational speed [rpm]',
    'Torque [Nm]',
    'Tool wear [min]'
]

X = df[features].copy()
y = df['Machine failure'].astype(int).copy()

# One-Hot Encoding
X = pd.get_dummies(X, columns=['Type'], drop_first=True)

# Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

print("Shapes -> X_train:", X_train.shape, "| X_test:", X_test.shape)
print("Proporção de falhas - treino:", y_train.mean().round(4), "| teste:", y_test.mean().round(4))
X_train.head()



## Modelagem e métricas
Treinaremos dois modelos e avaliaremos explicitamente as seguintes métricas:

- Accuracy: % de acertos no total. Pode enganar em dados desbalanceados.  
- Precision: dos que o modelo disse “falha”, quantos realmente eram falha.  
- Recall (sensibilidade): dos que eram falha, quantos o modelo detectou (evita falso negativo).  
- F1-score: média harmônica de Precision e Recall (equilíbrio).  
- ROC-AUC: capacidade de ranquear positivos acima dos negativos.  
- RMSE (educacional aqui): raiz do erro quadrático médio entre probabilidade prevista e rótulo real (0/1). Não é comum em classificação, mas ajuda a discutir erros de probabilidade.


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, ConfusionMatrixDisplay, classification_report,
    roc_auc_score, RocCurveDisplay
)
import numpy as np
import matplotlib.pyplot as plt

# Modelos
logreg = LogisticRegression(max_iter=200, class_weight='balanced', random_state=42)
rf = RandomForestClassifier(n_estimators=300, class_weight='balanced', n_jobs=-1, random_state=42)

logreg.fit(X_train, y_train)
rf.fit(X_train, y_train)

def avaliar_modelo(model, nome):
    print(f"\n=== {nome} ===")
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, digits=4))

    # Métricas explícitas (classe positiva = 1)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")

    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    ConfusionMatrixDisplay(cm).plot()
    plt.title(f"{nome} - Matriz de Confusão (Teste)")
    plt.show()

    # ROC-AUC e Curva ROC
    if hasattr(model, "predict_proba"):
        y_score = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_score)
        print(f"ROC-AUC: {auc:.4f}")
        RocCurveDisplay.from_predictions(y_test, y_score)
        plt.title(f"{nome} - Curva ROC (Teste)")
        plt.show()

        # RMSE educacional (probabilidade vs rótulo)
        rmse = np.sqrt(np.mean((y_score - y_test.values)**2))
        print(f"RMSE (probabilidade vs rótulo): {rmse:.4f}")
    else:
        print("Modelo não fornece predict_proba; AUC e RMSE não serão calculados.")

avaliar_modelo(logreg, "Regressão Logística")
avaliar_modelo(rf, "Random Forest")



### Compare os modelos
1) Qual modelo apresentou maior recall?  
2) Por que recall é importante para manutenção preditiva?  
3) Se você fosse gestor de TI, qual modelo implantaria e por quê?


Escreva suas respostas aqui


## Reflexão final do grupo (preenchimento obrigatório)
Dificuldades encontradas:  
- ...

O que aprendemos:  
- ...

Como isso se conecta à TI corporativa (servidores, roteadores, storage):  
- ...
